Импорт библиотек

In [67]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from Levenshtein import distance as lev

Cоздание функции, выполняющией расстояние Левенштейна

In [68]:
def levenstein(str_1, str_2):
    n, m = len(str_1), len(str_2)
    if n > m:
        str_1, str_2 = str_2, str_1
        n, m = m, n

    current_row = range(n + 1)
    for i in range(1, m + 1):
        previous_row, current_row = current_row, [i] + [0] * n
        for j in range(1, n + 1):
            add, delete, change = previous_row[j] + 1, current_row[j - 1] + 1, previous_row[j - 1]
            if str_1[j - 1] != str_2[i - 1]:
                change += 1
            current_row[j] = min(add, delete, change)

    return current_row[n]

1. Самое обычное сравнение

Реализация с помощью fuzzywuzzy

In [69]:
a = fuzz.ratio('Привет мир', 'Привет мир')
print(a)

100


In [70]:
a = fuzz.ratio('Привет мир', 'Привт привет')
print(a)

64


Реализация с помощью функции levenstein

In [71]:
levenstein('Привет мир', 'Привет мир')

0

In [72]:
levenstein('Привет мир', 'Привет привет')

5

2. Частичное сравнение


In [73]:
a = fuzz.partial_ratio('Привет мир', 'Привет привет!')
print(a)

80


In [74]:
a = fuzz.partial_ratio('Привет мир', 'Люблю колбасу, Привет мир')
print(a)

100


Реализация с помощью функции levenstein

In [85]:
def sim_lev(str1, str2):
    a = str1.split()
    b = str1.split()
    return levenstein(''.join(a), ''.join(b))

sim_lev('Привет мир', 'Люблю колбасу, Привет мир')

0

3. Сравнение по токену 

3.1. Token Sort Ratio

In [76]:
a = fuzz.token_sort_ratio('Привет наш мир', 'мир наш Привет')
print(a)

100


In [77]:
a = fuzz.token_sort_ratio('Привет наш мир', 'мир наш любимый Привет')
print(a)

78


In [78]:
a = fuzz.token_sort_ratio('1 2 Привет наш мир', '1 мир наш 2 ПриВЕт')
print(a)

100


Реализация с помощью функции levenstein

In [96]:
def token_lev(str1, str2):
    a = set(str1.split())
    b = set(str1.split())
    return levenstein(''.join(a), ''.join(b))

lev('Привет наш мир', 'мир наш Привет')

10

3.2. Token Set Ratio

In [79]:
a = fuzz.token_set_ratio('Привет наш мир', 'мир мир наш наш наш ПриВЕт')
print(a)

100


In [91]:
token_lev('Привет наш мир', 'мир мир наш наш наш ПриВЕт')

0

4. Продвинутое обычное сравнение

In [92]:
a = fuzz.WRatio('Привет наш мир', '!ПриВЕт наш мир!')
print(a)

100


In [93]:
a = fuzz.WRatio('Привет наш мир', '!ПриВЕт, наш мир!')
print(a)

97


Реализация с помощью функции levenstein

In [98]:
def default_plus_lev(str1, str2):
    a = str1.lower()
    b = str2.lower()
    return levenstein(a, b)

default_plus_lev('Привет наш мир', '!ПриВЕт, наш мир!')

3

5. Работа со списком

In [82]:
city = ["Москва", "Санкт-Петербург", "Саратов", "Краснодар", "Воронеж", "Омск", "Екатеринбург", "Орск", "Красногорск", "Красноярск", "Самара"]
a = process.extract("Саратов", city, limit=2)
# Параметр limit по умолчанию имеет значение 5
print(a)

[('Саратов', 100), ('Самара', 62)]


In [101]:
def find_city(str, list):
    list = []
    result = {}
    for i in list:
        if str == list[i]:
            result[list[i]] = 0
        elif lev(str, list[i]) < 10:
            result[list[i]] = lev(str, list[i])
    return result

find_city("Саратов", city)

{}

In [83]:
city = ["Москва", "Санкт-Петербург", "Саратов", "Краснодар", "Воронеж", "Омск", "Екатеринбург", "Орск", "Красногорск", "Красноярск", "Самара"]
a = process.extractOne("Краногрск", city)
print(a)

('Красногорск', 90)


fuzzywuzzy показывает в процентах схожесть текста друг с другом, функция levenstein() показывает, сколько символов нужно заменить